# Redis basic

In [ ]:
import redis
from redis.exceptions import ResponseError
from time import perf_counter as clock, sleep
from threading import Thread

## 1. Make connection

### 1.1. Connect with default connection pool

In [ ]:
rd = redis.Redis(host='localhost', port=6379, db=0, decode_responses=True)

### 1.2. Create connection pool and make connection

In [ ]:
pool = redis.ConnectionPool(host='localhost', port=6379, decode_responses=True)
rd = redis.Redis(host='localhost', port=6379, connection_pool=pool, decode_responses=True)

## 2. String

### 2.1. Set, get and delete key

- `SET key value`
- `GET key` -> `value`
- `DEL key`

In [ ]:
r = rd.set(name='a', value='ok')
print('* the result of [set a ok] is: {}'.format(r))

r = rd.get(name='a')
print('* the value of key "a" is: "{}"'.format(r))

r = rd.delete('a')
print('* the result of [delete a] is: {}'.format(r))

r = rd.get(name='a')
print('* after delete key "a", the value of key "a" is: {}'.format(r))

### 2.2. Set if exist or not exist

#### 2.2.1. Set key if exist

- `SET key value XX`

In [ ]:
try:
    r = rd.set(name='a', value='ok', xx=True)
    print('* when key "a" not exist, the result of [set a ok xx] is: {}'.format(r))

    r = rd.set(name='a', value='yes')
    print('* the result of [set a yes] is: {}'.format(r))

    r = rd.set(name='a', value='ok', xx=True)
    print('* when key "a" exist, the result of [set a ok xx] is: {}'.format(r))

    r = rd.get(name='a')
    print('* the value of key "a" is: "{}"'.format(r))
finally:
    rd.delete('a')

#### 2.2.1. Set key if not exist

- `SET key value NX`

In [ ]:
try:
    r = rd.set(name='a', value='ok', nx=True)
    print('* when key "a" not exist, the result of [set a ok nx] is: {}'.format(r))

    r = rd.get(name='a')
    print('* the value of key "a" is: "{}"'.format(r))

    r = rd.set(name='a', value='yes', nx=True)
    print('* when key "a" exist, the result of [set a yes nx] is: {}'.format(r))

    r = rd.get(name='a')
    print('* the value of key "a" is: "{}"'.format(r))
finally:
    rd.delete('a')

### 2.3. Key expire

#### 2.3.1. Expire by seconds

- `SET key value EX seconds`

In [ ]:
import time

try:
    r = rd.set(name='a', value='ok', ex=2)
    print('* the result of [set a ok ex 2] is {}'.format(r))

    r = rd.get(name='a')
    print('\n* before key "a" is expired, the value of key "a" is: {}'.format(r))

    r = rd.pttl(name='a')
    print('* the PTTL of key "a" is: {}'.format(r))

    time.sleep(2)

    r = rd.get(name='a')
    print('\n* after key "a" is expired, the value of key "a" is: {}'.format(r))

    r = rd.pttl(name='a')
    print('* the PTTL of key "a" is: {}'.format(r))
finally:
    rd.delete('a')

#### 2.3.1. Expire by milliseconds

- `SET key value PX milliseconds`

In [ ]:
import time

try:
    r = rd.set(name='a', value='ok', px=2000)
    print('* the result of [set a ok px 2000] is {}'.format(r))

    r = rd.get(name='a')
    print('\n* before key "a" is expired, the value of key "a" is: {}'.format(r))

    r = rd.pttl(name='a')
    print('* the PTTL of key "a" is: {}'.format(r))

    time.sleep(2)

    r = rd.get(name='a')
    print('\n* after key "a" is expired, the value of key "a" is: {}'.format(r))

    r = rd.pttl(name='a')
    print('* the PTTL of key "a" is: {}'.format(r))
finally:
    rd.delete('a')

### 2.4. Get and set value at sametime

- `GETSET key newvalue` -> `oldvalue`

In [ ]:
try:
    rd.set(name='a', value='ok')
    
    r = rd.get(name='a')
    print('* the value of key "a" is: {}'.format(r))

    r = rd.getset(name='a', value='yes')
    print('* the result of [getset a yes] is: {}'.format(r))

    r = rd.get(name='a')
    print('* the value of key "a" is: "{}"'.format(r))
finally:
    rd.delete('a')

### 2.5. Range operator

#### 2.5.1. Set value by range

- `SETRANGE key offset value` -> `length`

In [ ]:
try:
    rd.set(name='a', value='abcdefgh')
    
    r = rd.get(name='a')
    print('* set key "a", result is: "{}"'.format(r))

    r = rd.setrange(name='a', offset=3, value='DEF')
    print('* the result of [setrange a 3 DEF] is: {}'.format(r))

    r = rd.get(name='a')
    print('* the value of key "a" is: "{}"'.format(r))
finally:
    rd.delete('a')

#### 2.5.2. Get value by range

- `GETRANGE key start end` -> `substring`

In [ ]:
try:
    rd.set(name='a', value='123456789')
    
    r = rd.get(name='a')
    print('* the value of key "a" is: {}'.format(r))

    r = rd.getrange(key='a', start=2, end=5)
    print('* the result of [getrange a 2 5] is: {}'.format(r))
finally:
    rd.delete('a')

### 2.6. Get length of value

- `STRLEN key` -> `length`

In [ ]:
try:
    rd.set(name='a', value='123456789')
    
    r = rd.get(name='a')
    print('* the value of key "a" is: {}'.format(r))

    r = rd.strlen(name='a')
    print('* the result of [strlen a] is: {}'.format(r))
finally:
    rd.delete('a')

### 2.7. Append to value

- `APPEND key value` -> `length`

In [ ]:
try:
    r = rd.append(key='a', value='10')
    print('* when key "a" not exist, result of [append a 1] is: {}'.format(r))

    r = rd.append(key='a', value='20')
    print('* when key "a" exist, result of [append a 2] is: {}'.format(r))

    r = rd.append(key='a', value='30')
    print('* when key "a" exist, result of [append a 3] is: {}'.format(r))

    r = rd.get(name='a')
    print('* the value of key "a" is: "{}"'.format(r))
finally:
    rd.delete('a')

### 2.8. Increase and decrease number value

#### 2.8.1. Plus integer value

- `INCR key` -> `result`
- `INCRBY key value` -> `result`

In [ ]:
try:
    rd.set(name='a', value=1)
    
    r = rd.get(name='a')
    print('* the value of key "a" is: {}'.format(r))

    r = rd.incr(name='a')
    print('\n* the result of [incr a] is: {}'.format(r))

    r = rd.get(name='a')
    print('  the value of key "a" is: {}'.format(r))
    
    r = rd.incr(name='a', amount=2)
    print('\n* the result of [incr a 2] is: {}'.format(r))
    
    r = rd.get(name='a')
    print('  the value of key "a" is: {}'.format(r))
    
    r = rd.incr(name='a', amount=4)
    print('\n* the result of [incrby a 4] is: {}'.format(r))
    
    r = rd.get(name='a')
    print('  the value of key "a" is: {}'.format(r))
finally:
    rd.delete('a')

#### 2.8.2. Plus float value

- `INCRBYFLOAT key value` -> `result`

In [ ]:
try:
    rd.set(name='a', value=1)
    
    r = rd.get(name='a')
    print('* the value of key "a" is: {}'.format(r))

    r = rd.incrbyfloat(name='a', amount=0.1)
    print('\n* the result of [incrbyfloat a 0.1] is: {}'.format(r))

    r = rd.get(name='a')
    print('  the value of key "a" is: {}'.format(r))
finally:
    rd.delete('a')

#### 2.8.3. Minus integer value

- `DECR key` -> `result`
- `DECRBY key value` => `result`

In [ ]:
try:
    rd.set(name='a', value=2)

    r = rd.get('a')
    print('* the value of key "a" is: {}'.format(r))

    r = rd.decr(name='a')
    print('\n* the result of [decr a] is: {}'.format(r))

    r = rd.get(name='a')
    print('  the result of key "a" is: {}'.format(r))

    r = rd.decr(name='a', amount=1)
    print('\n* the result of [decr a 1] is: {}'.format(r))

    r = rd.get(name='a')
    print('  the result of key "a" is: {}'.format(r))

    r = rd.decrby(name='a', amount=1)
    print('\n* the result of [decrby a 1] is: {}'.format(r))

    r = rd.get(name='a')
    print('  the result of key "a" is: {}'.format(r))
finally:
    rd.delete('a')

### 2.9. Get and set multi-items

- `MSET key1 value1 key2 value2 ...`
- `MGET key1 key2 ...` -> `[value1, value2, ...]`

In [ ]:
try:
    r = rd.mset(mapping={
        'a': 1,
        'b': 2,
        'c': 3
    })
    print('* the result of [set a 1 b 2 c 3] is: {}'.format(r))

    r = rd.mget('a', 'b', 'c')
    print('* the values [mget a b c] are: {}'.format(r))
finally:
    rd.delete('a')

### 2.10. Iterator

In [ ]:
for n in range(1, 1000):
    rd.set('k_{}'.format(n), n)

#### 2.10.1. Find all keys

- `KEYS` -> `[keys ...]`

In [ ]:
r = rd.keys()
print('* there are {} keys in db'.format(len(r)))

#### 2.10.2. Find all keys by pattern

- `KEYS pattern` -> `[keys ...]`

In [ ]:
r = rd.keys(pattern='k_12*')
print('* there are {} keys in db by pattern "k_12*"'.format(len(r)))

print('* keys are: {}'.format(r))

#### 2.10.3. Scan keys

- `SCAN cursor [COUNT count]` -> `[items ...]`

In [ ]:
cursor = 0

result_set = set()

print('* iterator of all keys:')

while True:
    r = rd.scan(cursor=cursor, count=100)
    result_set.update(r[1])

    print('\tcursor {} to {}, content is: {}, total is: {}'.format(cursor, r[0], len(r[1]), len(result_set)))

    cursor = r[0]
    if cursor == 0:
        break

#### 2.10.4. Scan keys by pattern

- `SCAN cursor [MATCH pattern] [COUNT count]` -> `[items ...]`

In [ ]:
cursor = 0

result_set = set()

print('* iterator of all keys:')

while True:
    r = rd.scan(cursor=cursor, count=100, match='k_12*')
    result_set.update(r[1])

    print('\tcursor {} to {}, content is: {}, total is: {}'.format(cursor, r[0], len(r[1]), len(result_set)))

    cursor = r[0]
    if cursor == 0:
        break
        
print('* scan result is: {}'.format(result_set))

#### 2.10.5. Clear all key

In [ ]:
rd.flushall()

## 3. Hash

### 3.1. Set and get in hash

#### 3.1.1. Set and get hash value with field

- `HSET hash field value`
- `HGET hash field` -> `value`

In [ ]:
try:
    r = rd.hset(name='a', key='k1', value='v1')
    print('* the result of [hset a k1 v1] is: {}'.format(r))

    r = rd.hget(name='a', key='k1')
    print('  the value of [hget a k1] is: {}'.format(r))

    rd.delete('a')

    r = rd.hset(name='a', mapping={'k1': 'v1', 'k2': 'v2'})
    print('\n* the result of [hset a k1 v1 k2 v2] is: {}'.format(r))

    r = rd.hget(name='a', key='k1')
    print('  the value of [hget a k1] is: {}'.format(r))

    r = rd.hget(name='a', key='k2')
    print('  the value of [hget a k2] is: {}'.format(r))
finally:
    rd.delete('a')

#### 3.1.2. Set hash field if it not exist

- `HSETNX key field value`
- `HDEL key field`

In [ ]:
try:
    rd.hset(name='a', key="k1", value='v1')

    r = rd.hsetnx(name='a', key='k1', value='v1_1')
    print('* when key exist, the result of [setnx a k1 v1] is: {}'.format(r))

    r = rd.hget(name='a', key='k1')
    print('  the value of [hget a k1] is: {}'.format(r))

    r = rd.hdel('a', 'k1')
    print('\n* the result of [hdel a k1] is: {}'.format(r))

    r = rd.hsetnx(name='a', key='k1', value='v1_1')
    print('\n* when key not exist, the result of [setnx a k1 v1_1] is: {}'.format(r))

    r = rd.hget(name='a', key='k1')
    print('  the value of [hget a k1] is: {}'.format(r))
    
finally:
    rd.delete('a')

## 4. List

### 4.1. Push and pop

#### 4.1.1. Push into list

- `LPUSH key value [value …]` -> `length`
- `RPUSH key value [value …]` -> `length`
- `LLEN key` -> `length`
- `LINDEX key index` -> `value`

In [ ]:
try:
    r = rd.lpush('a', *[(n + 1) * 10 for n in range(0, 10)])
        
    print('* length of list is: {}'.format(r))
    
    print('* content of list are: '.format(r))
    for i in range(0, rd.llen('a')):
        r = rd.lindex(name='a', index=i)
        print('\t({}, {})'.format(i, r))
        
    rd.delete('a')
    
    rd.rpush('a', *[(n + 1) * 10 for n in range(0, 10)])

    r = rd.llen(name='a')
    print('\n* length of list is: {}'.format(r))
    
    print('* content of list are: '.format(r))
    for i in range(0, rd.llen('a')):
        r = rd.lindex(name='a', index=i)
        print('\t({}, {})'.format(i, r))
finally:
    rd.delete('a')

#### 4.1.2. Pop from list

- `LPOP key` -> `value`
- `RPOP key` -> `value`

In [ ]:
try:
    r = rd.lpush('a', *[(n + 1) * 10 for n in range(0, 10)])
    
    print('* pop item form list one by one are: ')
    while rd.llen(name='a') > 0:
        r = rd.lpop(name='a')
        print('\tafter pop {}, and list count is {}'.format(r, rd.llen(name='a')))
        
    rd.lpush('a', *[(n + 1) * 10 for n in range(0, 10)])
    
    print('\n* pop item form list one by one are: ')
    while rd.llen(name='a') > 0:
        r = rd.rpop(name='a')
        print('\tafter pop {}, and list count is {}'.format(r, rd.llen(name='a')))
finally:
    rd.delete('a')

#### 4.1.3. Push if exist

- `LPUSHX key value` -> `length`
- `RPUSHX key value` -> `length`

In [ ]:
try:
    r = rd.lpushx('a', 100)
    print('* when key "a" not exist, [lpushx] return {}'.format(r))
    
    rd.lpush('a', 200)
    
    r = rd.lpushx('a', 100)
    print('* when key "a" exist, [lpushx] return {}'.format(r))
    
    r = rd.rpushx('a', 300)
    print('* when key "a" exist, [rpushx] return {}'.format(r))
finally:
    rd.delete('a')

#### 4.1.4. Push and push between two lists

- `RPOPLPUSH source destination`: Pop item from end of list 1 and push it in head of list 2

In [ ]:
try:
    len = rd.rpush('a', 1, 2, 3, 4)
    
    while rd.llen(name='a') > 0:
        rd.rpoplpush(src='a', dst='b')
        print('* list "a" is: {}, and list "b" is: {}'.format(rd.lrange('a', 0, len), rd.lrange('b', 0, len)))
finally:
    rd.delete('a', 'b')

### 4.2. Set, get and delete

#### 4.2.1. Get by range

- `LRANGE key start stop` -> `[item...]`

In [ ]:
try:
    r = rd.lpush('a', *[(n + 1) * 10 for n in range(0, 10)])
    
    r = rd.lrange(name='a', start=0, end=10)
    print('* get range({}, {}) of list: {}'.format(0, 10, r))
finally:
    rd.delete('a')

#### 4.2.2. Set item into list

- `LSET key index value`

In [ ]:
try:
    try:
        rd.lset(name='a', index=0, value='new')
    except ResponseError as err:
        print('* when list "a" empty, [lset] raise error: "{}"'.format(err))

    rd.lpush('a', 'old')
    print('* push into list: {}'.format(rd.lrange(name='a', start=0, end=0)))

    rd.lset(name='a', index=0, value='new')
    print('* when list "a" not empty, result of [lset] is: {}'.format(rd.lrange(name='a', start=0, end=0)))
finally:
    rd.delete('a')

#### 4.2.3. Insert item into list

- `LINSERT key BEFORE|AFTER pivot value`

In [ ]:
try:
    r = rd.linsert(name='a', where='BEFORE', refvalue=10, value=100)
    print('* before src value not exist, result of [linsert a BEFORE 10 100] is: {}'.format(r))

    r = rd.lpush('a', 10)
    print('* push into list: {}'.format(rd.lrange(name='a', start=0, end=r)))

    r = rd.linsert(name='a', where='BEFORE', refvalue=10, value=100)
    print('* after src value exist, result of [linsert a BEFORE 10 100] is: {}'.format(rd.lrange(name='a', start=0, end=r)))
    
    r = rd.linsert(name='a', where='AFTER', refvalue=10, value=200)
    print('* after src value exist, result of [linsert a AFTER 10 200] is: {}'.format(rd.lrange(name='a', start=0, end=r)))
finally:
    rd.delete('a')

#### 4.2.4. Remove item

- `LREM key count value`

In [ ]:
try:
    r = rd.rpush('a', *[5 - n if 5 - n > 0 else n - 5 for n in range(0, 11)])
    print('* list content is: \t\t\t{}'.format(rd.lrange(name='a', start=0, end=r)))

    r = rd.lrem(name='a', count=1, value=4)
    print('* remove {} items, then list is: \t{}'.format(r, rd.lrange(name='a', start=0, end=rd.llen('a'))))
    
    r = rd.rpush('b', *[5 - n if 5 - n > 0 else n - 5 for n in range(0, 11)])
    print('\n* list content is: \t\t\t{}'.format(rd.lrange(name='b', start=0, end=r)))

    r = rd.lrem(name='b', count=2, value=4)
    print('* remove {} items, then list is: \t{}'.format(r, rd.lrange(name='b', start=0, end=rd.llen('b'))))
finally:
    rd.delete('a', 'b')

### 4.3. Blocked queue

#### 4.3.1. Blocked pop

- `BLPOP key [key …] timeout` -> `item`
- `BRPOP key [key …] timeout` -> `item`

In [ ]:
try:
    start = clock()
    r = rd.blpop('a', timeout=2)
    print('* when list is not exist, [blpop] spend time {} sec, and result is: {}'.format(round(clock() - start, 2), r))
    
    rd.lpush(name='a', 100, 200)
    
    start = clock()
    r = rd.blpop('a', timeout=2)
    print('* when list is not empty, [blpop] spend time {} sec, and result is: {}'.format(round(clock() - start, 2), r))
    
    start = clock()
    r = rd.blpop('a', timeout=2)
    print('* when list is not empty, [blpop] spend time {} sec, and result is: {}'.format(round(clock() - start, 2), r))
    
    ####
    
    start = clock()
    r = rd.brpop('a', timeout=2)
    print('\n* when list is empty, [blpop] spend time {} sec, and result is: {}'.format(round(clock() - start, 2), r))
    
    rd.lpush('a', 100, 200)
    
    start = clock()
    r = rd.brpop('a', timeout=2)
    print('* when list is not empty, [blpop] spend time {} sec, and result is: {}'.format(round(clock() - start, 2), r))
    
    start = clock()
    r = rd.brpop('a', timeout=2)
    print('* when list is not empty, [blpop] spend time {} sec, and result is: {}'.format(round(clock() - start, 2), r))
finally:
    rd.delete('a')

#### 4.3.2. Blocked pop and push between two lists

- `BRPOPLPUSH source destination timeout`

In [ ]:
try:
    def append_list(seconds, items):
        print('* list appending thread started')
        sleep(seconds)
        rd.lpush('a', *items)

    t = Thread(target=append_list, args=(2, [1, 2, 3, 4]))
    t.start()

    start = clock()
    rd.brpoplpush(src='a', dst='b')
    print('* time spend {}, list "a" is: {}, and list "b" is: {}'.format(
        round(clock() - start, 2),
        rd.lrange(name='a', start=0, end=4),
        rd.lrange(name='b', start=0, end=4)))

    start = clock()
    rd.brpoplpush(src='a', dst='b')
    print('* time spend {}, list "a" is: {}, and list "b" is: {}'.format(
        round(clock() - start, 2),
        rd.lrange(name='a', start=0, end=4),
        rd.lrange(name='b', start=0, end=4)))

    t.join()
finally:
    rd.delete('a', 'b')

## 5. Set

### 5.1. Set and get

#### 5.1.1. Append and list item

- `SADD key member [member ...]` -> `length`
- `SMEMBERS key` -> `[member ...]`

In [ ]:
try:
    r = rd.sadd('a', 1, 2, 3, 4, 5)
    print('* there are {} items append into set "a"'.format(r))
    
    r = rd.sadd('a', *[n for n in range(10, 20)])
    print('* there are {} items append into set "a"'.format(r))
    
    r = rd.smembers(name='a')
    print('* the items in set "a" is: {}'.format(r))
finally:
    rd.delete('a')

#### 5.1.2. Test if containt

- `SISMEMBER key member` -> `bool`

In [ ]:
try:
    r = rd.sadd('a', *[n for n in range(10, 20)])
    print('* there are {} items append into set "a"'.format(r))
    
    r = rd.sismember(name='a', value=12)
    print('* the items "{}" in set "a" is: {}'.format(12, r))
    
    r = rd.sismember(name='a', value=22)
    print('* the items "{}" in set "a" is: {}'.format(22, r))
finally:
    rd.delete('a')

#### 5.1.3. Get item by random

- `SRANDMEMBER key [count]` -> `[items ...]`

In [ ]:
try:
    r = rd.sadd('a', *[n for n in range(10, 20)])
    print('* there are {} items append into set "a"'.format(r))
    
    r = rd.srandmember(name='a', number=5)
    print('* get 5 items by random are: "{}"'.format(r))
    
    r = rd.srandmember(name='a', number=5)
    print('* get 5 items by random are: "{}"'.format(r))
finally:
    rd.delete('a')

##### 5.1.4. Get count of items

- `SCARD key` -> `length`

In [ ]:
try:
    r = rd.scard(name='a')
    print('* when set "a" is empty, count of items is: {}'.format(r))
    
    r = rd.sadd('a', *[n for n in range(10, 20)])
    r = rd.scard(name='a')
    print('* after set "a" was filled, count of items is: {}'.format(r))
finally:
    rd.delete('a')

### 5.2. Pop, move and delete

#### 5.2.1. Pop item by random

- `SPOP key` -> `value`

In [ ]:
try:
    count = rd.sadd('a', *[n for n in range(10, 20)])

    while count > 0:
        r = rd.spop(name='a')
        count -= 1
        
        print('* item "{}" was poped, and left {} items'.format(r, count))
finally:
    rd.delete('a')

#### 5.2.2. Delete items

- `SREM key member [member …]`

In [ ]:
try:
    rd.sadd('a', *[n for n in range(10, 20)])

    r = rd.srem('a', 1)
    print('* remove not exist item, result is: {}, left items are: {}'.format(r, rd.smembers(name='a')))

    r = rd.srem('a', 10)
    print('* remove not exist item, result is: {}, left items are: {}'.format(r, rd.smembers(name='a')))

    r = rd.srem('a', 11, 12, 13, 14)
    print('* remove not exist item, result is: {}, left items are: {}'.format(r, rd.smembers(name='a')))
finally:
    rd.delete('a')

#### 5.2.3. Move item between to sets

- `SMOVE source destination member`

In [ ]:
try:
    rd.sadd('a', *[n for n in range(10, 20)])

    r = rd.smove(src='a', dst='b', value=1)
    print('* result of move result is: {}, '
          'set "a" is: {} and set "b" is: {}'.format(r, rd.smembers(name='a'), rd.smembers(name='b')))

    r = rd.smove(src='a', dst='b', value=10)
    print('* result of move result is: {}, '
          'set "a" is: {} and set "b" is: {}'.format(r, rd.smembers(name='a'), rd.smembers(name='b')))

    r = rd.smove(src='a', dst='b', value=15)
    print('* result of move result is: {}, '
          'set "a" is: {} and set "b" is: {}'.format(r, rd.smembers(name='a'), rd.smembers(name='b')))
finally:
    rd.delete('a')

### 5.3. Iterator

- `SSCAN key cursor [MATCH pattern] [COUNT count]` -> `[items ...]`

#### 5.3.1. Scan by key

In [ ]:
try:
    rd.sadd('a', *[n for n in range(1, 1000)])
    
    cursor = 0
    result_set = set()
    
    print('* iterator of key "a":')
    
    while True:
        r = rd.sscan(name='a', cursor=cursor, count=100)
        result_set.update(r[1])
        
        print('\tcursor from {} to {}, got {} items, total {}'.format(cursor, r[0], len(r[1]), len(result_set)))
        
        cursor = r[0]
        if cursor == 0:
            break
finally:
    rd.delete('a')

#### 5.3.2. Scan by pattern

In [ ]:
try:
    rd.sadd('a', *[n for n in range(1, 1000)])
    
    cursor = 0
    result_set = set()
    
    print('* iterator of key "a":')
    
    while True:
        r = rd.sscan(name='a', cursor=cursor, count=100, match='12*')
        result_set.update(r[1])
        
        print('\tcursor from {} to {}, got {} items, total {}'.format(cursor, r[0], len(r[1]), len(result_set)))
        
        cursor = r[0]
        if cursor == 0:
            break
            
    print('* the result of scan is: {}'.format(result_set))
finally:
    rd.delete('a')

In [ ]:
rd.connection_pool.disconnect()